# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f77a860ef10>
├── 'a' --> tensor([[ 0.2352,  1.8256,  1.5618],
│                   [ 0.0571,  0.2170, -2.1923]])
└── 'x' --> <FastTreeValue 0x7f77a860e9a0>
    └── 'c' --> tensor([[-0.5775, -0.0528, -0.8623,  1.7479],
                        [-0.4966,  0.2127,  1.1874, -0.4692],
                        [ 0.6018,  0.1297, -0.1041, -1.2848]])

In [4]:
t.a

tensor([[ 0.2352,  1.8256,  1.5618],
        [ 0.0571,  0.2170, -2.1923]])

In [5]:
%timeit t.a

68.7 ns ± 0.0271 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f77a860ef10>
├── 'a' --> tensor([[ 0.7365, -0.6917,  1.8769],
│                   [ 0.2519,  1.2031, -0.2722]])
└── 'x' --> <FastTreeValue 0x7f77a860e9a0>
    └── 'c' --> tensor([[-0.5775, -0.0528, -0.8623,  1.7479],
                        [-0.4966,  0.2127,  1.1874, -0.4692],
                        [ 0.6018,  0.1297, -0.1041, -1.2848]])

In [7]:
%timeit t.a = new_value

67.9 ns ± 0.8 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2352,  1.8256,  1.5618],
               [ 0.0571,  0.2170, -2.1923]]),
    x: Batch(
           c: tensor([[-0.5775, -0.0528, -0.8623,  1.7479],
                      [-0.4966,  0.2127,  1.1874, -0.4692],
                      [ 0.6018,  0.1297, -0.1041, -1.2848]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2352,  1.8256,  1.5618],
        [ 0.0571,  0.2170, -2.1923]])

In [11]:
%timeit b.a

59.7 ns ± 0.0461 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1881,  0.2054, -1.0151],
               [ 0.3463,  0.7112,  0.2917]]),
    x: Batch(
           c: tensor([[-0.5775, -0.0528, -0.8623,  1.7479],
                      [-0.4966,  0.2127,  1.1874, -0.4692],
                      [ 0.6018,  0.1297, -0.1041, -1.2848]]),
       ),
)

In [13]:
%timeit b.a = new_value

495 ns ± 3.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

823 ns ± 7.74 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 30.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 534 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 455 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f777ca87610>
├── 'a' --> tensor([[[ 0.2352,  1.8256,  1.5618],
│                    [ 0.0571,  0.2170, -2.1923]],
│           
│                   [[ 0.2352,  1.8256,  1.5618],
│                    [ 0.0571,  0.2170, -2.1923]],
│           
│                   [[ 0.2352,  1.8256,  1.5618],
│                    [ 0.0571,  0.2170, -2.1923]],
│           
│                   [[ 0.2352,  1.8256,  1.5618],
│                    [ 0.0571,  0.2170, -2.1923]],
│           
│                   [[ 0.2352,  1.8256,  1.5618],
│                    [ 0.0571,  0.2170, -2.1923]],
│           
│                   [[ 0.2352,  1.8256,  1.5618],
│                    [ 0.0571,  0.2170, -2.1923]],
│           
│                   [[ 0.2352,  1.8256,  1.5618],
│                    [ 0.0571,  0.2170, -2.1923]],
│           
│                   [[ 0.2352,  1.8256,  1.5618],
│                    [ 0.0571,  0.2170, -2.1923]]])
└── 'x' --> <FastTreeValue 0x7f77a8595250>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 62.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f76fb3384c0>
├── 'a' --> tensor([[ 0.2352,  1.8256,  1.5618],
│                   [ 0.0571,  0.2170, -2.1923],
│                   [ 0.2352,  1.8256,  1.5618],
│                   [ 0.0571,  0.2170, -2.1923],
│                   [ 0.2352,  1.8256,  1.5618],
│                   [ 0.0571,  0.2170, -2.1923],
│                   [ 0.2352,  1.8256,  1.5618],
│                   [ 0.0571,  0.2170, -2.1923],
│                   [ 0.2352,  1.8256,  1.5618],
│                   [ 0.0571,  0.2170, -2.1923],
│                   [ 0.2352,  1.8256,  1.5618],
│                   [ 0.0571,  0.2170, -2.1923],
│                   [ 0.2352,  1.8256,  1.5618],
│                   [ 0.0571,  0.2170, -2.1923],
│                   [ 0.2352,  1.8256,  1.5618],
│                   [ 0.0571,  0.2170, -2.1923]])
└── 'x' --> <FastTreeValue 0x7f76fb245970>
    └── 'c' --> tensor([[-0.5775, -0.0528, -0.8623,  1.7479],
                        [-0.4966,  0.2127,  1.1874, -0.4692],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.1 µs ± 72.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.5775, -0.0528, -0.8623,  1.7479],
                       [-0.4966,  0.2127,  1.1874, -0.4692],
                       [ 0.6018,  0.1297, -0.1041, -1.2848]],
              
                      [[-0.5775, -0.0528, -0.8623,  1.7479],
                       [-0.4966,  0.2127,  1.1874, -0.4692],
                       [ 0.6018,  0.1297, -0.1041, -1.2848]],
              
                      [[-0.5775, -0.0528, -0.8623,  1.7479],
                       [-0.4966,  0.2127,  1.1874, -0.4692],
                       [ 0.6018,  0.1297, -0.1041, -1.2848]],
              
                      [[-0.5775, -0.0528, -0.8623,  1.7479],
                       [-0.4966,  0.2127,  1.1874, -0.4692],
                       [ 0.6018,  0.1297, -0.1041, -1.2848]],
              
                      [[-0.5775, -0.0528, -0.8623,  1.7479],
                       [-0.4966,  0.2127,  1.1874, -0.4692],
                       [ 0.6018,  0.1297, -0.1041, -1.2848]],

In [26]:
%timeit Batch.stack(batches)

77.6 µs ± 60.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.5775, -0.0528, -0.8623,  1.7479],
                      [-0.4966,  0.2127,  1.1874, -0.4692],
                      [ 0.6018,  0.1297, -0.1041, -1.2848],
                      [-0.5775, -0.0528, -0.8623,  1.7479],
                      [-0.4966,  0.2127,  1.1874, -0.4692],
                      [ 0.6018,  0.1297, -0.1041, -1.2848],
                      [-0.5775, -0.0528, -0.8623,  1.7479],
                      [-0.4966,  0.2127,  1.1874, -0.4692],
                      [ 0.6018,  0.1297, -0.1041, -1.2848],
                      [-0.5775, -0.0528, -0.8623,  1.7479],
                      [-0.4966,  0.2127,  1.1874, -0.4692],
                      [ 0.6018,  0.1297, -0.1041, -1.2848],
                      [-0.5775, -0.0528, -0.8623,  1.7479],
                      [-0.4966,  0.2127,  1.1874, -0.4692],
                      [ 0.6018,  0.1297, -0.1041, -1.2848],
                      [-0.5775, -0.0528, -0.8623,  1.7479],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 7.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
